<a href="https://colab.research.google.com/github/Fawad433/TensorFlow-Projects/blob/master/C1/W4/assignment/C1_W4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Things to Note:**
1. When coding the `class myCallback`, Python 3 will run into an error
```python
TypeError: '>' not supported between instances of 'NoneType' and 'float'
```
when using the code
```python
if(logs.get('accuracy')>0.99):
```

For Python 3, use the following equivalent code line

```python
if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
```

2. You can run the notebook using TensorFlow 2.5.0

In [ ]:
#!pip install tensorflow==2.5.0

In [ ]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile
import numpy as np
from google.colab import files
from keras.preprocessing import image
!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

zip_ref = zipfile.ZipFile("./happy-or-sad.zip", 'r')
zip_ref.extractall("./h-or-s")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf
To: /content/happy-or-sad.zip
100% 2.67M/2.67M [00:00<00:00, 83.8MB/s]


In [6]:

# Directory with training horse pictures
train_happy_dir = os.path.join('./h-or-s/happy')
train_sad_dir = os.path.join('./h-or-s/sad')

In [7]:
print(f'total training horse images: {len(os.listdir(train_happy_dir))}')
print(f'total training human images: {len(os.listdir(train_sad_dir))}')

total training horse images: 40
total training human images: 40


In [37]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>DESIRED_ACCURACY):
          print("\nReached desired accuracy so cancelling training!")
          self.model.stop_training = True
        
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = 'relu', input_shape = (150,150,3)),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(3,3),
    #tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    #tf.keras.layers.MaxPooling2D(3,3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid'),

    ])

    model.summary()

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(lr = 0.001), metrics = ['accuracy'])
                 
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
                        './h-or-s',  # This is the source directory for training images
                        target_size=(150, 150),  # All images will be resized to 300x300
                        batch_size = 40,
                        # Since you use binary_crossentropy loss, you need binary labels
                        class_mode='binary')
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(train_generator,
                      steps_per_epoch=2,  
                      epochs=500,
                      verbose=1,
                      callbacks = [callbacks]
                       )
    
    ###predict
    uploaded = files.upload()

    for fn in uploaded.keys():
    
      # predicting images
      path = '/content/' + fn
      img = image.load_img(path, target_size=(150, 150))
      x = image.img_to_array(img)
      x /= 255
      x = np.expand_dims(x, axis=0)

      images = np.vstack([x])
      classes = model.predict(images, batch_size=10)
      print(classes[0])
      if classes[0]>0.5:
        print(fn + " is a happy")
      else:
        print(fn + " is a sad")
    
    return history.history['accuracy'][-1]

In [38]:
train_happy_sad_model()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 4, 4, 64)        

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/500
2/2 [==============================] - 1s 79ms/step - loss: 0.9214 - accuracy: 0.5000
Epoch 2/500
2/2 [==============================] - 0s 82ms/step - loss: 0.7352 - accuracy: 0.5750
Epoch 3/500
2/2 [==============================] - 0s 89ms/step - loss: 0.6279 - accuracy: 0.5000
Epoch 4/500
2/2 [==============================] - 0s 85ms/step - loss: 0.6166 - accuracy: 0.6375
Epoch 5/500
2/2 [==============================] - 0s 82ms/step - loss: 0.4526 - accuracy: 0.8250
Epoch 6/500
2/2 [==============================] - 0s 87ms/step - loss: 0.3769 - accuracy: 0.8750
Epoch 7/500
2/2 [==============================] - 0s 84ms/step - loss: 0.5798 - accuracy: 0.6875
Epoch 8/500
2/2 [==============================] - 0s 89ms/step - loss: 0.3006 - accuracy: 0.9000
Epoch 9/500
2/2 [==============================] - 0s 78ms/step - loss: 0.2482 - accuracy: 0.9250
Epoch 10/500
2/2 [==============================] - 0s 80ms/step - loss: 0.2000 - accuracy: 0.9375
Epoch 11/500
2/2 [=

Saving download.png to download (3).png
[0.2395722]
download.png is a sad


1.0